In [4]:
import pandas as pd
import requests
import warnings
import time
import multiprocessing
import threading
from concurrent.futures import ThreadPoolExecutor
from ipapy import is_valid_ipa
# from google.colab import files
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [7]:
df = pd.read_csv("ebanglalibrary_100k_with_ipa0_106500_validated_0_106500.csv")
df_words=pd.read_csv("cleanWords.csv")


In [299]:
# sort df_words by length of word
df=pd.read_csv("ipa_validation_batch1.xlsx - Sheet1.csv")
def length(word):
    if type(word) == str:
        return len(word)
    return 0
df_words['length'] = (df_words['words']).apply(length)
df_words = df_words.sort_values(by=['length'])
df_words = df_words.reset_index(drop=True)
df_words = df_words.drop(columns=['length'])
df_words = df_words.drop_duplicates(subset=['words'], keep='first')
df_words.to_csv('cleanWords.csv', index=False)

In [2]:
def replace_func(subset):
  for row in subset.itertuples():
    ipa=row.ipa+"%"
    ipa=ipa.replace("a","ɐ")
    ipa=ipa.replace("i","ɪ")
    ipa=ipa.replace("æ","ɛ")
    ipa=ipa.replace("r","ɾ")
    ipa=ipa.replace("ʲ ","e̯ ")
    for i, ch in enumerate(ipa):
      if ch=='ʲ':
        if ipa[i+1]==' ':
          ipa = ipa[:i] + "e̯" + ipa[i+1:]
        elif is_valid_ipa(ipa[i+1])==False:
          ipa = ipa[:i] + "e̯" + ipa[i+1:]

    ipa=ipa[:-1]
    df.at[row.Index,"validated_ipa"]=ipa

    

    
    # words=row.sentence.split(" ")
    # for word in words:
    #   if word[-1]=='ʲ':
    #     word=word[:-1] + "e̯"
        
    #   for ch in word:
        

In [5]:
df=pd.read_csv("ebanglalibrary_100k_with_ipa0_106500.csv")
start=0
steps=10000
end=106500
subsets=[]
start_pos=start
while start<end:
  subsets.append(df.iloc[start:start+steps])
  start=start+steps
pool=ThreadPoolExecutor(len(subsets))
ws=[]
for subset in subsets:
  ws.append(pool.submit(replace_func, subset))
print(pool.shutdown())

None


In [6]:
df.to_csv(f"ebanglalibrary_100k_with_ipa0_106500_validated_{start_pos}_{end}.csv",index=False)

In [8]:
prefix_sentence="__$$__"
prefix_words="__##__"
def replace_words(subset):
  all_sentences=""

  # combine all sentence from subset and replace all words from word list with special characters
  for row in subset.itertuples():
    if pd.notna(row.done):
      continue
    if all_sentences=="":
      all_sentences=row.sentence
    else:
      all_sentences=all_sentences+prefix_sentence+row.sentence
  for row in df_words.itertuples():
    if pd.isna(row.ɪpas) or pd.isna(row.words):
      continue
    # if pd.notna(row.word_fix):
    #   all_sentences=all_sentences.replace(" "+row.word_fix+" "," "+prefix_words+row.ɪpas+" ")
    all_sentences=all_sentences.replace(" "+row.words+" "," "+prefix_words+row.ɪpas+" ")
    all_sentences=all_sentences.replace(prefix_sentence+row.words+" ",prefix_sentence+prefix_words+row.ɪpas+" ")
    all_sentences=all_sentences.replace(" "+row.words+"।"," "+prefix_words+row.ɪpas+"।")
    all_sentences=all_sentences.replace(" "+row.words+"?"," "+prefix_words+row.ɪpas+"?")
    all_sentences=all_sentences.replace(" "+row.words+"!"," "+prefix_words+row.ɪpas+"!")
    all_sentences=all_sentences.replace(" "+row.words+","," "+prefix_words+row.ɪpas+",")
    all_sentences=all_sentences.replace(" "+row.words+":"," "+prefix_words+row.ɪpas+":")
    all_sentences=all_sentences.replace(" "+row.words+";"," "+prefix_words+row.ɪpas+";")
    all_sentences=all_sentences.replace(" "+row.words+"’"," "+prefix_words+row.ɪpas+"’")
    all_sentences=all_sentences.replace(" "+row.words+"”"," "+prefix_words+row.ɪpas+"”")
    all_sentences=all_sentences.replace("“"+row.words+" ","“"+prefix_words+row.ɪpas+" ")
    all_sentences=all_sentences.replace("("+row.words+" ","("+prefix_words+row.ɪpas+" ")
    all_sentences=all_sentences.replace("("+row.words+",","("+prefix_words+row.ɪpas+",")
    all_sentences=all_sentences.replace(" "+row.words+")"," "+prefix_words+row.ɪpas+")")
    
    
  all_sentences_arr=all_sentences.split(prefix_sentence);
  print(all_sentences_arr)



  for row,sentence_v in zip(subset.itertuples(),all_sentences_arr):
    if pd.notna(row.done):
      continue

    validated_ipa=row.validated_ipa.split(" ")
    new_ipa=sentence_v.split(" ")
    new_ipa_sentence=""
    if new_ipa[0]=="":
      new_ipa=new_ipa[1:]
    if validated_ipa[0]=="":
      validated_ipa=validated_ipa[1:]
    for word,valid in zip(validated_ipa, new_ipa):
      if valid.startswith(prefix_words):
        new_ipa_sentence=new_ipa_sentence+valid+" "
      else:
        new_ipa_sentence=new_ipa_sentence+word+" "
    
    
    
    # .replace(prefix_words,"")

    print(row.sentence)
    print(row.ipa)
    print(new_ipa_sentence)
    print("++++++++++++++++++++++++++++++++++++++")
    df.at[row.Index,"validated_ipa"]=new_ipa_sentence
    df.at[row.Index,"done"]="OK"
      
    # df.at[row.Index,"temp_sentence"]=all_sentences_arr[row.Index+1]

    

In [9]:
# test cell
df = pd.read_csv("ebanglalibrary_100k_with_ipa0_106500_validated_0_106500.csv")
df_words=pd.read_csv("cleanWords.csv")
replace_words(df.iloc[0:100])

['৯নব্যভারত (ষোড়শ খণ্ড, বৈশাখ-চৈত্র, ১৩০৫), ২৮০৷ __##__d̪oʃ __##__ɾobɪnd̪ɾonɐt̪ʰ __##__tʰɐkuɾ, __##__pɾoʃɔŋgo __##__kɔt̪ʰɐ।', '__##__t̪ɔbe __##__ɛk hɐɟɐɾ ɐtʃo bɐhɐt̪t̪oɾ __##__kʰɾɪʃtɐbd̪e বঙ্গদর্শন-এর __##__pɾokɐʃ __##__eɪ̯ ধারায় গতিবেগের __##__ʃɔncɐɾ ঘটায়।', '__##__bost̪ut̪ বাগ্মিতায় অক্ষয়কুমারের __##__cʰɪlo জন্মগত __##__od̪ʱɪkɐɾ।', 'ভারতীয় __##__ʃɪlpeɾ __##__kɔt̪ʰɐ __##__nɐmok পুস্তকে __##__ɪt̪ɪpuɾbe __##__ʃɪlpo সম্বন্ধীয় __##__t̪ɐ̃ɾ নিবন্ধগুলি __##__pɾokɐʃɪt̪ɔ।', '__##__onnɐnno বিষয়ের __##__ʃɔŋge __##__e পত্রিকায় __##__oɪ̯t̪ɪhɐʃɪk প্রবন্ধাবলিও __##__pɾokɐʃɪt̪ɔ __##__hot̪e __##__t̪ʰɐke।', '__##__ʃeɪ̯ __##__ot̪ɪt̪o __##__ɪt̪ɪhɐʃ __##__ɟekono পাশ্চাত্য __##__ʃɔbbʱot̪ɐ __##__ɔpekkʰɐ __##__kʰɔɾbo __##__nɔe̯, __##__eɪ̯ __##__pɾot̪ʰom __##__onud̪ʱɐbon __##__koɾl ভারতবাসী।', '__##__pɔɾe সাধনা-র __##__pɾokɐʃ __##__bond̪ʱo হয়ে __##__gele সিরাজদ্দৌল্লা-র __##__ɔboʃɪʃtɐŋʃo ভারতী-তে __##__pɾokɐʃɪt̪ɔ __##__hɔe̯।', '__##__ʃe __##__ɟonno __##__pɾɐcɪn __##__bɐŋlɐɾ __##__ɪt̪ɪhɐʃ রচনায় শিলা

In [11]:
# threadpool make sure everything is currect before running this cell
df = pd.read_csv("ebanglalibrary_100k_with_ipa0_106500_validated_with_words_no_space.csv")
start=0
steps=500
end=10000
subsets=[]
start_pos=start
while start<end:
  subsets.append(df.iloc[start:start+steps])
  start=start+steps
pool=ThreadPoolExecutor(len(subsets))
ws=[]
for subset in subsets:
  ws.append(pool.submit(replace_words, subset))
print(pool.shutdown())
df.to_csv(f"ebanglalibrary_100k_with_ipa0_106500_validated_with_words_no_space.csv",index=False)


['সে __##__ʃunt̪e __##__pɐbe __##__kɛno, __##__mɐt̪ʰɐɾ __##__upoɾ ম্যান্ডেলাদিদি উড়ে __##__oke ডাকছে।', '__##__t̪ɐɾɐ __##__ʃekʰɐne __##__bɐʃɐ বানায় __##__mukʰeɾ লালা __##__d̪ɪʲe।', '__##__ebɐɾe শতরঞ্জের __##__mɔt̪o পাহাড়তলি __##__bʱeʃe উঠেবে __##__cokʰe।', 'ম্যান্ডেলা __##__bɐbɐke জাদুকরের __##__kʰɔboɾ __##__d̪ɪt̪e সাহসই পায়নি। __##__ɐɾ সে-বারই __##__t̪o __##__bɐbɐ জাহাজডুবিতে __##__nɪkʰõɟ হয়ে __##__gelen।', '__##__mɐ __##__nɐ, __##__mɐmɐ __##__nɐ, __##__keu̯ __##__nɐ।', 'ছিপটার সুতো __##__cʰeɽe দিয়েছে—এবং __##__nɪl __##__hɐŋoɾeɾ গলায় __##__ɐtke __##__gɛcʰe __##__bɪʃɐl __##__ɛktɐ বঁড়শি।', '__##__ɟeno __##__bolcʰe, __##__ɐmɐɾ দু-চোয়ালে __##__kɛno __##__kɐtʰ __##__ɐtke __##__d̪ɪle।', '__##__t̪ɐɾ __##__ɟe __##__bʱɐlo চায় হাইতিতি __##__t̪ɐo̯ __##__boɟʱe। __##__ʃe __##__ɐɾ পাখিদের ঝাঁকের __##__pecʰone ধাওয়া __##__koɾe __##__nɐ।', '__##__tʰɪk __##__nɐlɪʃ __##__gɛcʰe __##__t̪ɐɾ __##__nɐme। __##__ʃe __##__t̪obu কিঞ্চিৎ __##__cupcɐp __##__t̪ʰeke __##__kɪ ভাবল-তারপর __##__bollo, ডাং 

In [ ]:
df

In [ ]:
for row in df.itertuples():
  if pd.isna(row.done):
    print(f"{row.Index} {row.sentence}")
  if row.Index>10001:
    break

In [249]:
df.to_csv(f"ebanglalibrary_100k_with_ipa0_106500_validated_{start_pos}_{end}.csv",index=False)

In [23]:
name="sourav ahmed"
# name[-1]="x"
name=name[:-1] + "x"
print(name.replace("a","A"))
print(name)

sourAv Ahmex
sourav ahmex


In [109]:
unique_char_bangla=[' ', '!', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~', '\xa0', '£', 'ª', '\xad', '¯', '·', 'Æ', 'â', '˃', 'ا', 'ث', 'خ', 'د', 'ر', 'س', 'ص', 'ع', 'ق', 'ل', 'ه', 'ي', '।', '॥', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'ঢ়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', 'ৱ', '৷', '৺', 'ਆ', '\u200c', '\u200d', '‒', '–', '—', '‘', '’', '“', '”', '•', '…', '−', '√', '∣', '⋯', '\uf020', '\uf02c', '\uf02d', '\uf05b', '\ufeff']
unique_char_ipa=[' ', '!', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~', '\xa0', '£', '\xad', 'æ', 'ŋ', 'ɔ', 'ɟ', 'ɽ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ', '̃', '̪', '̯', '͡', 'ا', 'ث', 'خ', 'د', 'ر', 'س', 'ص', 'ع', 'ق', 'ل', 'ه', 'ي', '।', '॥', '৷', '–', '—', '‘', '’', '“', '”', '•', '…', '∣', '\ufeff']

# for row in df.itertuples():
#   for ch in row.ipa:
#     if ch not in unique_char_ipa:
#       unique_char_ipa.append(ch)
# unique_char_ipa.sort()
# print(unique_char_ipa)

In [163]:
print(" ".join(unique_char_ipa))

  ! % ' ( ) * , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > ? A B C D E F G H I J K L M N O P Q R S T U V W Y [ ] _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z { | ~   £ ­ æ ŋ ɔ ɟ ɽ ʃ ʰ ʱ ʲ ʷ ̃ ̪ ̯ ͡ ا ث خ د ر س ص ع ق ل ه ي । ॥ ৷ – — ‘ ’ “ ” • … ∣ ﻿


MISC


In [ ]:
      # if ch=='ʲ' and (row.sentence[i+1]==" " or row.sentence[i+1]=="\n" or  row.sentence[i+1]=="।" or row.sentence[i+1]=="?" or row.sentence[i+1]=="!" or row.sentence[i+1]=="%" or ):


In [111]:
# for item1, item2, item3 in zip([1, 2, 3,], ['a', 'b', 'c','d'], [True, False, True,'d']):
#     print(item1, item2, item3)



    # print(row.sentence)
    # print(row.ipa)
    # print(new_ipa_sentence)
    # print("++++++++++++++++++++++++++++++++++++++")
    

1 a True
2 b False
3 c True


In [194]:
unique_words_test=[]
sub=df.iloc[2884:2994]
for row in sub.itertuples():
  for word in row.sentence.split(" "):
    if word not in unique_words_test:
      unique_words_test.append(word)


unique_words_test

['ধোয়াটের',
 'জলে',
 'ভেসে-আসা',
 'ভরাটের',
 'মাটি',
 'নয়,',
 'সে',
 'একটি',
 'নতুন',
 'নিঃসঙ্গ',
 'নদী।সিটি',
 'কলেজে',
 'লেকচারারের',
 'কাজ',
 'করত',
 'জীবনানন্দ।',
 'আর',
 'ভালো',
 'হয়ে',
 'উঠেই',
 'আবার',
 'লেগেছি',
 'জীবিকার্জনের',
 'সংগ্রামে।এ',
 'রুগীর',
 'পক্ষে',
 'দুমকার',
 'পথ',
 'তো',
 'সাধ্যাতীত।',
 'সেই',
 'অল্পজলের',
 'প্রেমে',
 'সর্বস্বান্ত',
 'হয়েছিল',
 'কখনো?',
 'যেন',
 'এই',
 'সংগ্রামসংকুল',
 'সংসারের',
 'জন্যে',
 'সংসারপলাতক।',
 'সময়',
 'নেই,',
 'লাস্ট',
 'ট্রাম',
 'চলে',
 'গেল',
 'বোধ',
 'হয়।পরদিন',
 'গল্পের',
 'বাকিটা',
 'আমরাও',
 'শুনতে',
 'পেলাম।',
 'ওটা',
 'তখনো',
 'যুগ',
 'যে-যুগে',
 'প্রায়',
 'প্রেমেরই',
 'সমান-সমান',
 'বন্ধুতার',
 'দাম',
 'ছিল—সেই',
 'একই',
 'বিরহোৎকণ্ঠ',
 'বন্ধুতা।',
 'তার',
 'আগে',
 'সাবিত্রীপ্রসন্নের',
 'আমলেই',
 'নৃপেন',
 'বিজলীতে',
 'নাট্যসমালোচনা',
 'লিখত।',
 'ইডেন',
 'হিন্দু',
 'হসটেলের',
 'বাথরুমে',
 'দরজা',
 'বন্ধ',
 'করে',
 'কার্বলিক',
 'এসিড',
 'খেয়ে',
 'কথক',
 'বিজয়',
 'সেনগুপ্ত',
 'আত্মহত্যা',
 'করেছে।দেখতে',
 'গিয়েছিলাম',
 'তাকে

In [195]:
for row in df_words.itertuples():
  if row.words in unique_words_test:
    print(row.words)

এর
বাড়িতে
অকালে
অটুট
অতীতে
অত্যধিক
অনেকখানি
অন্ত
অন্তরালে
অন্য
অর্থাৎ
আইনে
আকাশের
আগে
আছে
আজ
আজও
আত্মহত্যা
আত্মার
আত্মিক
আধুনিক
আনন্দিত
আনেন
আপনাদের
আপনার
আবার
আমরা
আমরাও
আমাকে
আমাদের
আমার
আমি
আমিও
আর
আরো
আলাপ
আলো
আশা
আশ্বাস
আসবে
আসল
আসে
ইডেন
উচিত
উচ্চ
উঠতে
উড়িয়ে
উত্তর
উৎস
উদীয়মান
উদ্দেশ্য
উদ্ভূত
উপন্যাস
উপর
উপস্থিত
উপায়
উপেক্ষা
এ
এই
এক
একই
একজন
একটা
একটি
একটিও
একদিন
একবার
একমাত্র
একসময়
এটি
এতগুলো
এমন
এমনটি
এমনি
এর
এরই
এল
এলেন
এসিড
এসে
এসেও
ঐক্য
ও
ওকে
ওটা
ওদের
ওর
ক
কখন
কখনো
কটা
কতটা
কতটুকুইবা
কথা
কথার
কপি
কবি
কবিতা
কবিতায়
কবে
করত
করতাম
করতে
করতেও
করল
করলাম
করলেন
করা
করুন
করে
করেই
কলকাতায়
কলেজে
কল্পনাও
কল্পনার
কল্লোল
কল্লোলের
কাউকে
কাউকেই
কাজ
কারণ
কি
কিনা
কিন্তু
কী
কে
কেউ
কেন
কেমন
কোঠায়
কোন
কোনো
কোলাকুলি
কৌশলে
ক্যামেরা
ক্লিক
ক্ষীণ
ক্ষেতের
ক্ষেত্রে
খাইয়ে
খাঁটি
খুঁজে
খুলে
খুশি
খেয়ে
গভীর
গর্ব
গলা
গল্প
গল্পের
গা
গিয়ে
গিয়েছিল
গিয়েছিলাম
গুরুবন্দনা
গেল
গোটা
গোল
গোলে
গ্রীষ্মের
ঘর
ঘরে
চক্ষু
চমৎকার
চলে
চারপাশের
চালিয়েছিলেন
চিঠি
চিঠির
চিনতে
চিনির
চুপ
চেয়ে
চেষ্টায়
চোখে
ছড়া
ছবিটি
ছবিটির
ছাত্রকে
ছাত্

In [1]:
for i, ch in enumerate(" t̪ɪnɪ ɟɐt̪ɪbʱed̪ mɐnɪt̪en nɐ, hɪnd̪u muʃɔlmɐnke ʃɔmobʱɐbe d̪ekʰɪt̪en o ʃɪʃʃod̪ɪgeɾ mod̪d̪ʱe hɪnd̪u-muʃɔlmɐn ʃɔkol ɟɐt̪ɪke͡ɪ̯ gɾohon koɾɪt̪en।"):
  print(f"{i}:{ch}")

0: 
1:t
2:̪
3:ɪ
4:n
5:ɪ
6: 
7:ɟ
8:ɐ
9:t
10:̪
11:ɪ
12:b
13:ʱ
14:e
15:d
16:̪
17: 
18:m
19:ɐ
20:n
21:ɪ
22:t
23:̪
24:e
25:n
26: 
27:n
28:ɐ
29:,
30: 
31:h
32:ɪ
33:n
34:d
35:̪
36:u
37: 
38:m
39:u
40:ʃ
41:ɔ
42:l
43:m
44:ɐ
45:n
46:k
47:e
48: 
49:ʃ
50:ɔ
51:m
52:o
53:b
54:ʱ
55:ɐ
56:b
57:e
58: 
59:d
60:̪
61:e
62:k
63:ʰ
64:ɪ
65:t
66:̪
67:e
68:n
69: 
70:o
71: 
72:ʃ
73:ɪ
74:ʃ
75:ʃ
76:o
77:d
78:̪
79:ɪ
80:g
81:e
82:ɾ
83: 
84:m
85:o
86:d
87:̪
88:d
89:̪
90:ʱ
91:e
92: 
93:h
94:ɪ
95:n
96:d
97:̪
98:u
99:-
100:m
101:u
102:ʃ
103:ɔ
104:l
105:m
106:ɐ
107:n
108: 
109:ʃ
110:ɔ
111:k
112:o
113:l
114: 
115:ɟ
116:ɐ
117:t
118:̪
119:ɪ
120:k
121:e
122:͡
123:ɪ
124:̯
125: 
126:g
127:ɾ
128:o
129:h
130:o
131:n
132: 
133:k
134:o
135:ɾ
136:ɪ
137:t
138:̪
139:e
140:n
141:।
